In [1]:
#Purpose: loading collections records from the Mars
import requests
import json
import pandas as pd
import numpy as np

In [2]:
url = 'https://mars.cyverse.org/thing'
res = requests.get(url)
collectionsInfo = json.loads(res.text)
collectionsInfo

{'status': [{'status': '200', 'count': 3027724},
  {'status': '500', 'count': 4},
  {'status': '503', 'count': 249}],
 'authority': [{'authority': 'GEOME', 'count': 228457},
  {'authority': 'OPENCONTEXT', 'count': 820272},
  {'authority': 'SESAR', 'count': 1784731},
  {'authority': 'SMITHSONIAN', 'count': 194517}]}

In [3]:
def getID(index, collection, limit):
    identifiers = []

    for offset in index:
        payload = {'offset':offset, 'limit': limit, 'status':200, 'authority': collection}
        url = f'https://mars.cyverse.org/thing/'

        res = requests.get(url, params=payload)
        try:
            res_json = json.loads(res.text)
            identifiers += pd.json_normalize(res_json['data'])['id'].tolist()
        except Exception as e:
            print('error:', e)
        
    return identifiers

In [4]:
def getInfo(method, identifier):
    payload = {'full': 'false', 'format': method}
    url = f'https://mars.cyverse.org/thing/{identifier}'

    res = requests.get(url, params=payload)
    if res.text != 'Internal Server Error':
        res_json = json.loads(res.text)
        return res_json
    return None

In [19]:
def convertToDf(id, method):
    result = pd.DataFrame()
    for i in id:
        try:
            df = pd.json_normalize(getInfo(method, i))
            df['uri'] = i
            result = result.append(pd.json_normalize(getInfo(method, i)), ignore_index=True)
        except Exception as e:
            print(e)
    return result

In [6]:
#SESAR records
subsetIndex = pd.DataFrame([i*1000  for i in range(int(collectionsInfo['authority'][2]['count']/1000))]).sample(n=50, random_state=1)[0].tolist()

In [7]:
identifiers_5000 = getID(subsetIndex, 'SESAR', 1000)
identifiers_5000 = pd.DataFrame(identifiers_5000).sample(n=5000, random_state=1)[0].tolist()

In [10]:
SESAR_5000_original = convertToDf(identifiers_5000, 'original')
SESAR_5000_core = convertToDf(identifiers_5000, 'core')

In [40]:
SESAR_5000_original.to_csv('../Collections_data/SESAR_5000_original.csv')
SESAR_5000_core.to_csv('../Collections_data/SESAR_5000_core.csv')

In [10]:
#openContext data
openContextIndex = pd.DataFrame([i*1000 for i in range(int(collectionsInfo['authority'][1]['count']/1000))]).sample(n=50, random_state=1)[0].tolist()

In [11]:
len(set(openContextIndex))

50

In [12]:
openContext_5000 = getID(openContextIndex, 'OPENCONTEXT', 1000)

error: Expecting value: line 1 column 1 (char 0)
error: Expecting value: line 1 column 1 (char 0)


In [14]:
len(set(openContext_5000))

48000

In [15]:
openContext_5000 = pd.DataFrame(openContext_5000).sample(n=5000, random_state=1)[0].tolist()

In [16]:
OPENCONTEXT_5000_original = convertToDf(openContext_5000, 'original')


In [20]:
OPENCONTEXT_5000_core = convertToDf(openContext_5000, 'core')

In [30]:
OPENCONTEXT_5000_original.head()

,uri,label,Creator,updated,latitude,longitude,published,Contributor,context uri,late bce/ce,...,context label,item category,project label,Has taxonomic identifier,Has anatomical identification,Temporal Coverage,Has type,Consists of,Has former or current keeper,Has current location
0,http://opencontext.org/subjects/F0226517-BF6B-...,8376.F73,[{'id': 'http://opencontext.org/persons/2664C6...,2020-03-28T09:44:57Z,37.668196,32.827191,2013-02-13T00:00:00Z,[{'id': 'http://opencontext.org/persons/00A61C...,http://opencontext.org/subjects/9FC1F4A2-7D19-...,-6200.0,...,Turkey/Çatalhöyük/Mound East/Area Bach/Unit 8376,Animal Bone,Çatalhöyük Zooarchaeology,"[{'id': 'http://www.gbif.org/species/359', 'la...",[{'id': 'http://purl.obolibrary.org/obo/UBERON...,NaN,NaN,NaN,NaN,NaN
1,http://opencontext.org/subjects/56ad8919-7efb-...,BSV-vert-ribs-1734,[{'id': 'http://opencontext.org/persons/0056f3...,2021-06-24T13:05:03Z,45.08968,10.87539,2015-10-21T01:31:32Z,NaN,http://opencontext.org/subjects/3d0e091d-fc60-...,-300.0,...,Italy/Forcello Bagnolo San Vito/Sector R18/Con...,Animal Bone,Animal remains from Forcello,NaN,NaN,[{'id': 'http://n2t.net/ark:/99152/p06v8w4g9dz...,NaN,NaN,NaN,NaN
2,http://opencontext.org/subjects/fecd90f7-c93b-...,83.0906,[{'id': 'http://opencontext.org/persons/265a12...,2021-06-26T11:47:28Z,31.13142,35.52837,2019-10-05T00:00:00Z,NaN,http://opencontext.org/subjects/d0b88a12-d4c4-...,-2550.0,...,Jordan/Numayra/Unit NE 2-1 (unit Rm 16)/Locus 14,Pottery,Early Bronze Age Numayra,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,http://opencontext.org/subjects/895D165C-CB43-...,Bone 1490,[{'id': 'http://opencontext.org/persons/67303C...,2020-03-28T14:57:19Z,38.46,27.35,2013-08-11T00:00:00Z,[{'id': 'http://opencontext.org/persons/67303C...,http://opencontext.org/subjects/0C34ECAC-1FD3-...,-6000.0,...,Turkey/Ulucak/Level Vc,Animal Bone,Zooarchaeology of Neolithic Ulucak,[{'id': 'http://eol.org/pages/32609438#gbif-su...,[{'id': 'http://purl.obolibrary.org/obo/UBERON...,NaN,NaN,NaN,NaN,NaN
4,http://opencontext.org/subjects/1FB92E7F-FE45-...,Finds Bag 18,[{'id': 'http://opencontext.org/persons/21_DT_...,2017-02-10T21:43:29Z,37.830874,40.812707,2012-03-28T00:00:00Z,[{'id': 'http://opencontext.org/persons/E114A4...,http://opencontext.org/subjects/12EEA506-2C33-...,-900.0,...,Turkey/Kenan Tepe/Area C/Trench 4/Locus 4084,Sample,Kenan Tepe,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
OPENCONTEXT_5000_core.head()

,$schema,@id,label,sampleidentifier,description,hasContextCategory,hasMaterialCategory,hasSpecimenCategory,informalClassification,keywords,...,producedBy.samplingSite.label,producedBy.samplingSite.location.elevation,producedBy.samplingSite.location.latitude,producedBy.samplingSite.location.longitude,producedBy.samplingSite.placeName,curation.label,curation.description,curation.accessConstraints,curation.curationLocation,curation.responsibility
0,../../iSamplesSchemaBasic0.2.json,metadata/28722/k2cj89z1b,8376.F73,ark:/28722/k2cj89z1b,early bce/ce: -6500.0 | late bce/ce: -6200.0 |...,[Site of past human activities],[Biogenic non organic material],[Organism part],[Mammalia],"[Turkey, Çatalhöyük, Mound East, Area Bach, Un...",...,None,None,37.668196,32.827191,"[Turkey, Çatalhöyük, Mound East, Area Bach, Un...",Not Provided,Not Provided,Not Provided,Not Provided,Not Provided
1,../../iSamplesSchemaBasic0.2.json,metadata/28722/k24x5mn4z,BSV-vert-ribs-1734,ark:/28722/k24x5mn4z,early bce/ce: -600.0 | late bce/ce: -300.0 | u...,[Site of past human activities],[Biogenic non organic material],[Organism part],[],"[Italy, Forcello Bagnolo San Vito, Sector R18,...",...,None,None,45.08968,10.87539,"[Italy, Forcello Bagnolo San Vito, Sector R18,...",Not Provided,Not Provided,Not Provided,Not Provided,Not Provided
2,../../iSamplesSchemaBasic0.2.json,metadata/28722/k29316r81,83.0906,ark:/28722/k29316r81,early bce/ce: -2850.0 | late bce/ce: -2550.0 |...,[Site of past human activities],[Anthropogenic material],[Artifact],[],"[Jordan, Numayra, Unit NE 2-1 (unit Rm 16), Lo...",...,None,None,31.13142,35.52837,"[Jordan, Numayra, Unit NE 2-1 (unit Rm 16), Lo...",Not Provided,Not Provided,Not Provided,Not Provided,Not Provided
3,../../iSamplesSchemaBasic0.2.json,metadata/28722/k2rf5nh2c,Bone 1490,ark:/28722/k2rf5nh2c,early bce/ce: -6500.0 | late bce/ce: -6000.0 |...,[Site of past human activities],[Biogenic non organic material],[Organism part],[Sheep or goat],"[Turkey, Ulucak, Level Vc]",...,None,None,38.46,27.35,"[Turkey, Ulucak, Level Vc]",Not Provided,Not Provided,Not Provided,Not Provided,Not Provided
4,../../iSamplesSchemaBasic0.2.json,metadata/http://n2t.net/ark:/28722/k2vt1nm8t,Finds Bag 18,http://n2t.net/ark:/28722/k2vt1nm8t,early bce/ce: -1050.0 | late bce/ce: -900.0 | ...,[Site of past human activities],[Anthropogenic material],[Other solid object],[],"[Turkey, Kenan Tepe, Area C, Trench 4, Locus 4...",...,None,None,37.830874,40.812707,"[Turkey, Kenan Tepe, Area C, Trench 4, Locus 4...",Not Provided,Not Provided,Not Provided,Not Provided,Not Provided


In [22]:
OPENCONTEXT_5000_original.to_csv('../Collections_data/OPENCONTEXT_5000_original.csv')
OPENCONTEXT_5000_core.to_csv('../Collections_data/OPENCONTEXT_5000_core.csv')
